### Finetuning Code

Packages: 

datasets
transformers
peft
LoRa

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
from transformers import TrainingArguments, Trainer

# 1. Load the base model and tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v0.6"  # or another base
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. Load and process your product QA dataset
dataset = load_dataset("json", data_files="product_qa.jsonl")

def format_example(example):
    # Example: put instruction/prompt and response together in text
    # Adjust tokens or special tokens as needed
    return {
      "input_ids": tokenizer(
          example["instruction"] + example["input"],
          truncation=True,
          max_length=512
      )["input_ids"],
      "labels": tokenizer(
          example["output"], 
          truncation=True,
          max_length=512
      )["input_ids"],
    }

dataset = dataset["train"].map(format_example)
dataset = dataset.remove_columns(["instruction", "input", "output"])

# 3. Set up LoRA fine-tuning
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],  # depends on model architecture
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

# 4. Training Arguments
training_args = TrainingArguments(
    output_dir="lora-fine-tuned",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    logging_steps=50,
    save_steps=500,
    evaluation_strategy="no"  # or "steps" if you have a val set
)

# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# 6. Fine-Tune
trainer.train()

# 7. Save your LoRA adapters + tokenizer
trainer.model.save_pretrained("lora-fine-tuned")
tokenizer.save_pretrained("lora-fine-tuned")